In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSouite
import numpy as np
import spacy
from checklist.perturb import Perturb


In [2]:
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
model = model_wrapper.ModelWrapper()
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)


In [3]:
editor = checklist.editor.Editor()
editor.tg

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
qs = []
labels = []
all_questions = set()
for x in open('/home/marcotcr/datasets/glue/glue_data/QQP/dev.tsv').readlines()[1:]:
    try:
        q1, q2, label = x.strip().split('\t')[3:]
    except:
        print(x)
        continue
    all_questions.add(q1)
    all_questions.add(q2)
    qs.append((q1, q2))
    labels.append(label)
labels = np.array(labels).astype(int)

"	What does the Quran say about homosexuality?	0



In [6]:
# [x for x in qs if 'better' in x[0]]

In [7]:
import pickle
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_qqp.pkl', 'rb'))

In [8]:
parsed_qs = [(spacy_map[q[0]], spacy_map[q[1]]) for q in qs]

In [9]:
suite = TestSuite()

## Vocabulary

In [10]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
print(', '.join(professions))

journalist, historian, nurse, engineer, accountant, attorney, architect, artist, editor, actor, actress, analyst, intern, economist, entrepreneur, author, assistant, interpreter, investor, executive, organizer, escort, educator, agent, academic, activist, advisor, administrator, investigator, auditor


In [11]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
# professions

In [12]:
print(', '.join(editor.suggest('{first_name} {last_name} is a good {mask}.')[:30]))

guy, player, writer, actor, example, man, person, friend, kid, hitter, read, shooter, one, coach, dude, pick, teacher, shot, fighter, artist, student, quarterback, poet, character, reader, sport, name, bet, cook, reporter


In [13]:
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))

In [14]:
print(', '.join(editor.suggest('Is {first_name} {last_name} {a:mask} {noun}?', noun=nouns)[:50]))

good, actual, excellent, real, important, American, angel, effective, decent, successful, better, public, true, great, serious, independent, old, evil, experienced, elite, competent, outstanding, ideal, bad, accomplished, FBI, qualified, sound, abusive, international, professional, trained, credible, legitimate, amazing, proper, trustworthy, Australian, career, honest, gifted, corrupt, certified, state, modern, accurate, political, private, brilliant, genuine


In [15]:
adjs = ['effective', 'actual', 'American', 'active', 'honest', 'excellent', 'elite', 'acomplished', 'official', 'outstanding', 'experienced', 'independent', 'international', 'aspiring', 'average', 'good', 'amazing', 'exceptional', 'successful', 'accredited', 'English', 'real', 'bad', 'terrible', 'fake', 'unusual', 'influential', 'incompetent']

In [16]:
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} {a:adj} {noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0, name='Modifier: adj', capability='Vocabulary', 
          description = 'Adding an adjective makes questions non-duplicates')
# test.run(new_pp)
# test.summary(n=6)
suite.add(test)

Is person {a1, a2}?

In [17]:
print(', '.join(editor.suggest('Is John Wayne {mask}?')))
print()
print(', '.join(editor.suggest('Is John Wayne {a:mask}?')[:50]))

dead, gay, alive, right, Dead, real, Jewish, mad, wrong, correct, insane, back, crazy, related, OK, Right, immortal, evil, straight, interested, famous, racist, sane, next, President, alone, lying, relevant, okay, DEAD, bisexual, God, angry, guilty, ok, Alone, Wrong, ready, gone, innocent, King, human, Muslim, Black, assassinated, Batman, here, Real, dying, involved, homosexual, special, Famous, happy, free, murdered, Superman, president, joking, White, Back, Gay, Catholic, cool, safe, resurrected, killed, American, missing, corrupt, cursed, alright, single, Crazy, Mad, Satan, Missing, legit, shot, Evil, finished, worthy, doomed, sincere, aware, autistic, responsible, dangerous, Christian, Cool, Immortal, radioactive, remembered, Next, serious, cheating, different, popular, Legend, listening, now, illegitimate, homophobic, kidding, sick, smart, connected, trustworthy, Innocent, legal

racist, atheist, outlaw, asshole, actor, ape, American, inspiration, idiot, orphan, Christian, ancesto

In [18]:
adjs_without_overlap = ['dead', 'gay', 'Jewish', 'Christian', 'American', 'mad', 'immortal', 'evil', 'famous', 'racist', 'Muslim', 'white', 'black', 'English', 'autistic', 'Australian', 'trustworthy', 'an atheist', 'an anarchist', 'an inventor', 'Indian', 'Armenian', 'an astronaut', 'an immigrant']

In [19]:
t = editor.template((
    'Is {first_name} {last_name} {adj1}?',
    'Is {first_name} {last_name} {adj2}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='different adjectives', capability = 'Vocabulary',
          description='Same first and last name, different adjectives')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

person1 and person2 are different by first and last name

In [20]:
t = editor.template((
    'Is {first_name1} {last_name1} {adj}?',
    'Is {first_name2} {last_name2} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people', capability = 'Vocabulary',
          description='Different first and last name, same adjectives')
test.run(new_pp)
test.summary(n=5)
suite.add(test)

Predicting 974 examples
Test cases:      974
Fails (rate):    63 (6.5%)

Example fails:
0.6 ('Is James Powell evil?', 'Is Nathan Gray evil?')
----
0.8 ('Is Anna Stewart white?', 'Is Laura Howard white?')
----
0.7 ('Is Angela Bennett an inventor?', 'Is Christina Walker an inventor?')
----
1.0 ('Is James Diaz evil?', 'Is Jonathan Rodriguez evil?')
----
1.0 ('Is Justin Davis American?', 'Is Jeremy Butler American?')
----


person1 and person2 are different by first name only

In [21]:
t = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name2} {last_name} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people v2', capability = 'Vocabulary',
          description='Different first name, same adjective and last name')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

person1 and person2 are different by last name only

In [22]:
t = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name} {last_name2} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people v3', capability = 'Vocabulary',
          description='Different last name, same adjective and first name')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

different animals

In [23]:
print(', '.join(editor.suggest('I have a pet {mask}.')))

cat, dog, rabbit, turtle, too, spider, now, rat, name, goat, lizard, dragon, named, pig, wolf, monkey, girl, squirrel, owl, also, here, called, there, tiger, python, friend, lab, phone, mix, bear, bird, …, boy, farm, duck, snake, zoo, owner, problem, fish, carrier, Shiva, shop, myself, companion, boyfriend, kitten, store, dinosaur, animal, watch, ring, project, deer, Labrador, Shepherd, tree, killer, lobster, mom, dish, X, bunny, girlfriend, shark, already, gun, car, puppy, one, computer, elephant, fetish, bug, brother, lover, tattoo, monitor, snail, cow, XD, Goat, mouse, somewhere, robot, sometimes, mother, syndrome, haha, pair, knife, life, carriage, Persian, cats, collar, that, diary, door, family, boo, number, show, assistant, toy, horse, chicken, laptop, server, tracker


In [24]:
animals = ['cat', 'dog', 'rabbit', 'turtle', 'spider', 'rat', 'goat', 'lizard', 'pig', 'monkey', 'squirrel', 'owl', 'snake', 'fish', 'lobster', 'snail', 'chicken']

In [25]:
print(', '.join(editor.suggest('Can I feed my {an} {mask}?', an=animals)))

food, pellets, eggs, water, again, worms, properly, now, meat, today, too, anything, outside, here, something, tonight, poop, fish, this, milk, directly, that, myself, rice, instead, nuts, there, live, steak, more, dinner, formula, treats, poison, regularly, correctly, rabbit, right, blood, enough, back, chicken, cheese, once, larvae, peanuts, feed, alive, honey, grass, feces, soup, anyway, scraps, bugs, what, alone, breakfast, oil, raw, well, chocolate, urine, corn, cereal, sugar, seeds, home, liver, free, better, protein, online, daily, cookies, carrots, lunch, yogurt, candy, cat, grain, seed, spinach, anymore, salad, fat, antibiotics, greens, Rice, it, indoors, …, shit, inside, money, toys, crap, organic, naturally, hay, bacon, some, cats, bird, bamboo, egg, breast, separately, rats, juice, vegetables, biscuits, waste, butter, crow, insulin, soy, sometimes, meal, mix, bones, yet, fly, appropriately, balls, privately, bananas, tuna, salmon, chickens, Milo, birds, bread, saliva, chick

In [26]:
food = ['eggs', 'water',  'worms', 'meat', 'poop', 'milk', 'rice', 'nuts', 'steak', 'formula',  'soup', 'bugs', 'oil', 'chocolate', 'corn', 'cereal', 'sugar', 'seeds', 'liver', 'cookies', 'carrots', 'yogurt', 'salad', 'greens', 'rice', 'bananas', 'tuna', 'apples', 'salmon', 'butter', 'insulin', 'soy']

In [27]:
t = editor.template((
    'Can I feed my {animal1} {food}?',
    'Can I feed my {animal2} {food}?',
    ),
    animal=animals,
    food=food,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Different animals' 
desc = 'Ask the same question about two different pet animals, expect prediction to be 0'
test = MFT(**t, labels=0, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

Modifiers that don't matter

In [28]:
action = editor.suggest('Is that {animal} really {mask} on the couch?', animal=animals)[:30]

editor.suggest('Is that {animal} {mask} {action} on the couch?', animal=animals, action=action)
non_changing_modifier = ['really', 'truly', 'actually', 'indeed', 'in fact', 'currently', 'literally', 'somehow']
t = editor.template((
    'Is that {animal} {action} on the {place}?',
    'Is that {animal} {mod2} {action} on the {place}?',
    ),
    action=action,
    animal=animals,
    mod=non_changing_modifier,
    place =['couch','bed', 'sofa', 'table'],
    remove_duplicates=False, 
    nsamples=1000)
name = 'Irrelevant modifiers - animals' 
desc = 'Add modifiers that preserve question semantics (e.g. \'really\')'
test = MFT(**t, labels=1, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

In [29]:
action = editor.suggest('Is {first_name1} {mask} to {first_name2}?')[:30]
editor.suggest('Is {first_name1} {mask} {a} to {first_name2}?', a=action)
non_changing_modifier = ['really', 'truly', 'actually', 'indeed', 'in fact']
t = editor.template((
    'Is {first_name1} {action} to {first_name2}?',
    'Is {first_name1} {mod2} {action} to {first_name2}?',
    ),
    action=action,
    mod=non_changing_modifier,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Irrelevant modifiers - people' 
desc = 'Add modifiers that preserve question semantics (e.g. \'really\')'
test = MFT(**t, labels=1, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [30]:
t = editor.template((
    'My pet {animal1} eats {food}. Is it normal for animals to eat {food}?',
    'My pet {animal2} eats {food}. Is it normal for animals to eat {food}?',
    ),
    animal=animals,
    food=food,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Irrelevant preamble with different examples.' 
desc = 'Ask questions about animals in general, but with different examples in the preamble. Expect duplicate.'
test = MFT(**t, labels=1, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [31]:
editor.visual_suggest('I broke my {mask} playing tennis.')

TemplateEditor(bert_suggests=['arm', 'leg', 'wrist', 'back', 'hand', 'neck', 'foot', 'ankle', 'nose', 'thumb',…

In [32]:
# nouns = editor.selected_suggestions
nouns = ['finger', 'forearm', 'feet', 'head', 'foot', 'elbow', 'nose', 'back', 'heart', 'shoulder', 'skull', 'toe', 'hip', 'neck', 'body', 'thumb', 'rib', 'knee', 'jaw', 'heel', 'thigh', 'ankle', 'arm', 'wrist', 'eye', 'spine', 'leg', 'butt', 'teeth', 'skin', 'bone', 'hand', 'face']

In [33]:
t = editor.template((
    'I hurt my {n} last time I played {sport}. {followup}',
    'I hurt my {n2} last time I played {sport}. {followup}',
    ),
    n=nouns,
    sport=['tennis', 'golf', 'soccer', 'football'],
    followup=['Is this going to impact my performance?',
              'Should I never play again?',
              'Is this a common injury?',
             'Is it normal to hurt this part of the body?',],
    remove_duplicates=True, 
    nsamples=1000)
name = 'Preamble is relevant (different injuries)' 
desc = 'Question preamble mentions different injuries, which makes the questions themselves non-duplicates.'
test = MFT(**t, labels=0, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

### Taxonomy

Antonyms

In [34]:
tmp = []
x = editor.suggest('How can I become more {mask}?')
x += editor.suggest('How can I become less {mask}?')
for a in set(x):
    e = editor.synonyms('How can I become {moreless} %s?' % a, a, moreless=['more', 'less'])
    if e:
#         print(a, [b[0][0] for b in e] )
        tmp.append([a] + e)
#         opps.append((a, e[0][0][0]))
print(',\n'.join([str(tuple(x)) for x in tmp]))

('fat', 'productive', 'rich', 'fatty'),
('progressive', 'liberal', 'imperfect'),
('enlightened', 'educated', 'clear'),
('extremist', 'radical'),
('shy', 'timid', 'unsure'),
('religious', 'spiritual'),
('confident', 'positive'),
('alienated', 'alien', 'estranged'),
('important', 'significant', 'authoritative'),
('passive', 'peaceful', 'inactive'),
('aware', 'mindful'),
('inspired', 'divine'),
('consistent', 'uniform', 'logical', 'coherent'),
('biased', 'bias'),
('unhappy', 'distressed'),
('stressed', 'stress'),
('emotional', 'excited'),
('frustrated', 'queer', 'defeated', 'disappointed'),
('controlling', 'moderate', 'control'),
('corrupt', 'corrupted'),
('disconnected', 'confused', 'fragmented'),
('free', 'liberal', 'innocent'),
('efficient', 'effective'),
('organized', 'organised', 'direct'),
('mean', 'average', 'hateful'),
('kind', 'tolerant'),
('disruptive', 'troubled'),
('addicted', 'addict'),
('rude', 'primitive', 'crude'),
('mindful', 'aware'),
('committed', 'attached'),
('authori

In [35]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'),
            ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'),
            ('humble', 'modest'), ('courageous', 'brave'), ('happy', 'joyful'), ('scared', 'frightened'),
           ]

In [36]:

t = editor.template([
    (
    'How can I become more {x[0]}?',
    'How can I become more {x[1]}?',
    ),
    (
    'How can I become more {x[1]}?',
    'How can I become more {x[0]}?',
    ),
    (
    'How can I become less {x[0]}?',
    'How can I become less {x[1]}?',
    ),
    (
    'How can I become less {x[1]}?',
    'How can I become less {x[0]}?',
    ),
    (
    'How can I become {a:x[0]} person?',
    'How can I become {a:x[1]} person?',
    ),
    (
    'How can I become {a:x[1]} person?',
    'How can I become {a:x[0]} person?',
    ),
],
    unroll=True,
    x=synonyms,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become more {synonym}?' 
desc = 'different (simple) templates where words are replaced with their synonyms'
test = MFT(**t, labels=1, name=name, capability = 'Taxonomy',
          description=desc)
test.run(new_pp)
test.summary(n=3)
suite.add(test, overwrite=True)

Predicting 6000 examples
Test cases:      6000
Fails (rate):    1248 (20.8%)

Example fails:
0.0 ('How can I become an outspoken person?', 'How can I become a vocal person?')
----
0.0 ('How can I become less vocal?', 'How can I become less outspoken?')
----
0.0 ('How can I become an outspoken person?', 'How can I become a vocal person?')
----


In [37]:

import re
def replace_pairs(pairs):
    def replace_z(text):
        ret = []
        for x, y in pairs:
            t = re.sub(r'\b%s\b' % x, y, text )
            if t != text:
                ret.append(t)
            if y == 'smart':
                continue
            t = re.sub(r'\b%s\b' % y, x, text )
            if t != text:
                ret.append(t)
        return list(set(ret))
    return replace_z
def apply_and_pair(fn):
    def ret_fn(text):
        ret = fn(text)
        return [(text, r) for r in ret]
    return ret_fn

In [38]:
synonyms

[('spiritual', 'religious'),
 ('angry', 'furious'),
 ('organized', 'organised'),
 ('vocal', 'outspoken'),
 ('grateful', 'thankful'),
 ('intelligent', 'smart'),
 ('humble', 'modest'),
 ('courageous', 'brave'),
 ('happy', 'joyful'),
 ('scared', 'frightened')]

In [39]:
name = '(question, f(question)) where f(question) replaces synonyms?' 
desc = 'Expect 1, should be easy because it\'s individual word changes'
t = Perturb.perturb(list(all_questions), apply_and_pair(replace_pairs(synonyms)), nsamples=200, keep_original=False)
test = INV(t.data, threshold=0.1)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)


Predicting 200 examples
Test cases:      200
Fails (rate):    0 (0.0%)


In [40]:
def apply_to_each_and_product(fn):
    def apply_to_one(x):
        p = fn(x)
        if not p:
            p = []
        return list(set([x] + p))
    def ret_fn(pair):
        p1 = apply_to_one(pair[0])
        p2 = apply_to_one(pair[1])
        return [x for x in itertools.product(p1, p2) if x != pair]
    return ret_fn

In [41]:
name = 'Replace synonyms in real pairs'
desc = ''
t = Perturb.perturb(qs, apply_to_each_and_product(replace_pairs(synonyms)), nsamples=200, keep_original=True)
test = INV(t.data, threshold=0.1)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)

Predicting 548 examples
Test cases:      200
Fails (rate):    28 (14.0%)

Example fails:
0.9 ('Can Atheism solve problems created by religious fundamentalism?', 'Is atheism an answer to religious terrorism?')
0.0 ('Can Atheism solve problems created by religious fundamentalism?', 'Is atheism an answer to spiritual terrorism?')
0.1 ('Can Atheism solve problems created by spiritual fundamentalism?', 'Is atheism an answer to religious terrorism?')

----
0.8 ('Why did only human beings evolve so much compared to other animals?', 'Why are human beings so intelligent compared to all the other kinds of animals?')
0.2 ('Why did only human beings evolve so much compared to other animals?', 'Why are human beings so smart compared to all the other kinds of animals?')

----
0.9 ('What is the best advice for a happy life?', 'What is the best way of living life?')
0.0 ('What is the best advice for a joyful life?', 'What is the best way of living life?')

----


Antonyms

In [42]:
opps = []
x = editor.suggest('How can I become more {mask}?')
x += editor.suggest('How can I become less {mask}?')
for a in set(x):
    e = editor.tg.antonyms('How can I become more %s?' % a, a)
    if e:
#         print(a, e[0][0][0])
        opps.append((a, e[0][0][0]))

In [43]:
opps = []
x = editor.suggest('How can I become more {mask}?')
x += editor.suggest('How can I become less {mask}?')
for a in set(x):
    e = editor.antonyms('How can I become {moreless} %s?' % a, a, moreless=['more', 'less'])
    if e:
#         print(a, [b[0][0] for b in e] )
        opps.append([a] + e)
#         opps.append((a, e[0][0][0]))
print(','.join([str(tuple(x)) for x in opps]))

('fat', 'lean', 'thin'),('progressive', 'conservative'),('shy', 'confident'),('religious', 'secular'),('impatient', 'patient'),('passive', 'active'),('unhappy', 'happy'),('emotional', 'intellectual'),('organic', 'functional'),('rude', 'civil', 'polite'),('defensive', 'offensive'),('pessimistic', 'optimistic'),('courageous', 'fearful'),('capitalist', 'socialist'),('individual', 'common'),('cautious', 'brave'),('stupid', 'intelligent', 'smart'),('conservative', 'progressive', 'liberal'),('insecure', 'secure'),('visible', 'invisible'),('powerless', 'powerful'),('positive', 'negative', 'neutral'),('negative', 'positive'),('bad', 'good'),('irresponsible', 'responsible'),('specific', 'general'),('active', 'passive'),('conspicuous', 'invisible'),('smart', 'stupid'),('hungry', 'thirsty'),('optimistic', 'pessimistic'),('dependent', 'independent'),('hopeful', 'hopeless'),('humble', 'proud')


In [44]:
opps = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]

In [45]:

t = editor.template([(
    'How can I become more {x[0]}?',
    'How can I become less {x[0]}?',
    ),
    (
    'How can I become less {x[1]}?',
    'How can I become more {x[1]}?',
    )],
    unroll=True,
    x=opps,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become more X != How can I become less X' 
desc = ''
test = MFT(**t, labels=0, name=name, capability = 'Vocabulary',
          description=desc)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)

Predicting 2000 examples
Test cases:      2000
Fails (rate):    790 (39.5%)

Example fails:
0.9 ('How can I become less secular?', 'How can I become more secular?')
----
0.7 ('How can I become more irresponsible?', 'How can I become less irresponsible?')
----
0.6 ('How can I become more pessimistic?', 'How can I become less pessimistic?')
----


In [46]:

t = editor.template([(
    
    'How can I become more {x[0]}?',
    'How can I become less {x[1]}?',
    ),
    (
    'How can I become less {x[0]}?',
    'How can I become more {x[1]}?',
    )],
    unroll=True,
    x=opps,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become more X = How can I become less antonym(X)' 
desc = ''
test = MFT(**t, labels=1, name=name, capability = 'Taxonomy',
          description=desc)
test.run(new_pp)
test.summary(n=3)
# suite.add(test)

Predicting 2000 examples
Test cases:      2000
Fails (rate):    1349 (67.5%)

Example fails:
0.0 ('How can I become more active?', 'How can I become less passive?')
----
0.2 ('How can I become more religious?', 'How can I become less secular?')
----
0.0 ('How can I become less powerless?', 'How can I become more powerful?')
----


DIR version (kinda bad, won't add to suite)

In [47]:
t = Perturb.perturb(list(all_questions), apply_and_pair(replace_pairs(opps)), nsamples=200, keep_original=False)
test = DIR(t.data, expect=Expect.eq(0), agg_fn='all')
test.run(new_pp)
test.summary(n=5)
# suite.add(test)


Predicting 210 examples
Test cases:      200
Fails (rate):    80 (40.0%)

Example fails:
1.0 ('Should Scotland be independent? Would it be better off independent from the UK?', 'Should Scotland be dependent? Would it be better off dependent from the UK?')
----
0.9 ('Structural Engineering: What is negative moment /positive moment at supports/midspan?', 'Structural Engineering: What is negative moment /negative moment at supports/midspan?')
0.9 ('Structural Engineering: What is negative moment /positive moment at supports/midspan?', 'Structural Engineering: What is positive moment /positive moment at supports/midspan?')

----
1.0 ("Are there any telescopes powerful enough to view evidence left behind from NASA's moon landings?", "Are there any telescopes powerless enough to view evidence left behind from NASA's moon landings?")
----
0.8 ('Is it possible to get good trekking poles under 1.5k?', 'Is it possible to get bad trekking poles under 1.5k?')
----
0.8 ('What good vs. bad habits ha

## Robustness

In [48]:
def wrap_apply_to_each(fn, both=False, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, str(q2)) for x in fnq1])
        ret.extend([(str(q1), x) for x in fnq2])
        if both:
            ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn
def wrap_apply_to_both(fn, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn

Typos

In [49]:
t = Perturb.perturb(qs, wrap_apply_to_each(Perturb.add_typos), nsamples=500)
test = INV(t.data, name='add one typo', capability='Robustness', description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test, overwrite=True)

Contractions

In [50]:
t = Perturb.perturb(qs, wrap_apply_to_each(Perturb.contractions, both=True), nsamples=500)
test = INV(**t, name='contrations', capability='Robustness', description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

Paraphrases

In [51]:
import itertools
def me_to_you(text):
    t = re.sub(r'\bI\b', 'you', text)
    t = re.sub(r'\bmy\b', 'your', t)
    return re.sub(r'\bmine\b', 'yours', t)
def paraphrases(text):
    ts = ['How do I ', 'How can I ', 'What is a good way to ', 'How should I ']
    templates1 = ['How do I {x}?', 'How can I {x}?', 'What is a good way to {x}?', 'If I want to {x}, what should I do?',
                'In order to {x}, what should I do?']
    ts2 = ['Can you ', 'Can I ']#, 'Do I']
    ts3 = ['Do I ']
    templates2 = ['Can you {x}?', 'Can I {x}?', 'Do you think I can {x}?', 'Do you think you can {x}?',]
    templates3 = ['Do I {x}?', 'Do you think I {x}?']
    ret = []
    for i, (tsz, templates) in enumerate(zip([ts, ts2, ts3], [templates1, templates2, templates3])):
        for t in tsz:
            if text.startswith(t):
                x = text[len(t):].strip('?')
                ts = editor.template(templates, x=x).data[0]
                if i <= 1:
                    ts = ts + [me_to_you(x) for x in ts]
                ret += ts
    return ret
def paraphrases_product(text):
    pr = paraphrases(text)
    return list(itertools.product(pr, pr))

def paraphrase_each(pair):
    p1 = paraphrases(pair[0])
    p2 = paraphrases(pair[1])
    return list(itertools.product(p1, p2))

In [52]:
t = Perturb.perturb(list(all_questions), paraphrases_product, nsamples=200, keep_original=False)
name = '(q, paraphrase(q))'
desc = 'For questions that start with "How do I X", "How can I X", etc'
test = DIR(t.data, expect=Expect.eq(1), agg_fn='all', name=name, description=desc, capability='Robustness')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test, overwrite=True)

In [53]:
t = Perturb.perturb(qs, paraphrase_each, nsamples=100, keep_original=True)
name = 'Product of paraphrases(q1) * paraphrases(q2)'
desc = 'For questions that start with "How do I X", "How can I X", etc'
test = INV(t.data, name=name, description=desc, capability='Robustness')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)


## Ner

### Change same name, number, location in both

Names

In [54]:
def change_both_wrapper(fn):
    def change_both(qs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True)
        c2 = fn(q2, seed=seed, meta=True)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        return [(q1, q2) for q1, q2, m1, m2 in zip(c1, c2, m1, m2) if m1 == m2]
    return change_both

def change_each_wrapper(fn):
    def change_one(qs, **kwargs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True, **kwargs)
        c2 = fn(q2, seed=seed, meta=True, **kwargs)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        ret = []
        ret.extend([(q1_, str(q2)) for q1_, m1_ in zip(c1, m1) if m1_[0] in str(q2)])
        ret.extend([(str(q1), q2_) for q2_, m2_ in zip(c2, m2) if m2_[0] in str(q1)])
        return ret
    return change_one
    
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_names), nsamples=500)
test = INV(**t, name='Change same name in both questions', capability='NER',
          description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

Locs

In [55]:
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_location), nsamples=500)
test = INV(**t, name='Change same location in both questions', capability='NER',
          description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

Numbers

In [56]:
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_number), nsamples=500)
test = INV(**t, name='Change same number in both questions', capability='NER',
          description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

### Change name, loc, number in only one where orig prediction is duplicate

Changing only first names

In [57]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500, first_only=True)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_names), nsamples=500, first_only=True)
expect_fn = Expect.eq(0)
expect_fn = Expect.slice_orig(expect_fn, lambda orig, *args: orig == 1)
name = 'Change first name in one of the questions'
desc = 'Take pairs that are originally predicted as duplicates, change first name in one of them and expect new prediction to be non-duplicate'
test = DIR(**t, expect=expect_fn, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test)


Changing first and last names

In [58]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_names), nsamples=1500)
name = 'Change first and last name in one of the questions'
desc = 'Take pairs that are originally predicted as duplicates, change first and last name in one of them and expect new prediction to be non-duplicate'
test = DIR(**t, expect=expect_fn, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test)


Locs

In [59]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_location), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_location), nsamples=1500)
name = 'Change location in one of the questions'
desc = 'Take pairs that are originally predicted as duplicates, change location in one of them and expect new prediction to be non-duplicate'
test = DIR(**t, expect=expect_fn, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

numbers

In [60]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_number), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_number), nsamples=1500)
name = 'Change numbers in one of the questions'
desc = 'Take pairs that are originally predicted as duplicates, change number in one of them and expect new prediction to be non-duplicate'
test = DIR(**t, expect=expect_fn, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test, overwrite=True)

In [61]:
### Change 

### Keep entities, fill in with BERT gibberish

In [62]:
def mask_gibberish(question):
    ents = question.ents
    if not ents:
        return None
    wp = [x.text for x in question if x.tag_ in ['WP', 'WRB', 'WDT']]
    if not wp:
        wp = question[0].text
    else:
        wp = wp[0]
    ents = [x.text for x in ents]
    ents[-1] = ents[-1] + '?'
    template = ' {mask} '.join([wp] + ents)
    gibberish = editor.template(template).data[:5]
#     return gibberish
    ret = [(question.text, x) for x in gibberish if question.text.lower() != x.lower() ]
    return ret
def gibberish_both(qs):
    q1, q2 = qs
    ret = []
    x1 = mask_gibberish(q1)
    if x1:
        ret.extend(x1)
    x2 = mask_gibberish(q2)
    if x2:
        ret.extend(x2)
    return ret
#     ret =  [(question.text, x[1]) for x in tg.fill_in_between([wp] + ents) if question.text != x[1]]
#     ret = [x for x in ret if len(x[1].split()) < len(x[0].split()) -4][:5]
#     return rett

In [ ]:
t = Perturb.perturb(parsed_qs, gibberish_both, nsamples=500)
expect_false = Expect.eq(0)
name = 'Keep entitites, fill in with gibberish'
desc = 'Fill in between entitites with BERT, expect result to not be duplicate with original questions'
test = DIR(**t, expect=expect_false, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test, overwrite=True)

In [ ]:
# test.summary(3)

## Temporal

Is != used to be

In [ ]:
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Did {first_name} {last_name} use to be {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
name = 'Is person X != Did person use to be X'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Is != becoming

In [ ]:
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} becoming {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
name = 'Is person X != Is person becoming X'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Before != after

In [ ]:
t = editor.template((
    'What was {first_name} {last_name}\'s life before becoming {a:noun}?',
    'What was {first_name} {last_name}\'s life after becoming {a:noun}?'
),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
name = 'What was person\'s life before becoming X != What was person\'s life after becoming X'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
action12 = [x for x in editor.suggest('Do you have to {mask} your cat before {mask} it?') if 'kill' not in x[0] and 'kill' not in x[1]][:200]

In [ ]:
t = editor.template((
    'Do you have to {a[0]} your {an} before {a[1]} it?',
    'Do you have to {a[0]} your {an} after {a[1]} it?'
),
    an = ['cat', 'dog', 'hamster'],
    a=action12,
    remove_duplicates=True, 
   nsamples=1000)
name = 'Do you have to X your dog before Y it != Do you have to X your dog after Y it.'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
print(', '.join(editor.suggest('Is is {mask} to eat after 10pm?')))

In [ ]:
mid = ['normal', 'ok', 'safe', 'dangerous', 'acceptable', 'reasonable', 'proper', 'wrong', 'healthy', 'important']

In [ ]:
print(', '.join(editor.suggest('Is is {mid} to {mask} after 10pm?', mid=mid)))

In [ ]:
activity = ['drink', 'sleep', 'drive', 'work', 'eat', 'smoke', 'walk', 'read', 'party', 'talk', 'exercise', 'celebrate', 'text', 'tweet', 'run', 'dance', 'swim', 'cook', 'pray', 'pee', 'rest']

In [ ]:
t = editor.template(('Is it {mid} to {activity} before {hour}{ampm}?','Is it {mid} to {activity} after {hour}{ampm}?'),
                activity=activity,
                mid=mid,
                hour=[str(x) for x in range(1, 12)],
                ampm=['am', 'pm'],
                remove_duplicates=True, 
                nsamples=1000)
name = 'Is it {ok, dangerous, ...} to {smoke, rest, ...} after != before'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

## Negation

In [ ]:
mid2 = mid + ['legal', 'awkward', 'socially acceptable']

In [ ]:
print(', '.join(editor.suggest('Is it {mid} to {mask} in {country}?', mid=mid2)[:100]))

In [ ]:
things = ['work', 'vote', 'travel', 'marry', 'drive', 'study', 'protest', 'campaign', 'fight', 'gamble', 'hunt', 'pray', 'smoke', 'fish', 'murder', 'invest', 'pee', 'march', 'worship', 'volunteer', 'surf', 'shoot', 'dance', 'camp', 'preach', 'spy', 'be gay', 'lie', 'divorce', 'discriminate']

In [ ]:
t = editor.template(('Is it {mid} to {activity} in {country}?','Is it {mid} not to {activity} in {country}?'),
                activity=things,
                mid=mid2,
                remove_duplicates=True, 
                nsamples=1000)
name = 'Is it {ok, dangerous, ...} to {smoke, rest, ...} in country != Is it {ok, dangerous, ...} not to {smoke, rest, ...} in country'
test = MFT(**t, labels=0, name=name, description='', capability='Negation')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
t = editor.template((
    'What are things {a:noun} should worry about?',
    'What are things {a:noun} should not worry about?',
),
                noun=nouns,
                remove_duplicates=True, 
                nsamples=1000)
name = 'What are things a {noun} should worry about != should not worry about.'
test = MFT(**t, labels=0, name=name, description='', capability='Negation')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
t = editor.template([(
    'How can I become {a:x[0]} person?',
    'How can I become a person who is not {x[1]}?',
    ),
    (
    'How can I become {a:x[1]} person?',
    'How can I become a person who is not {x[0]}?',
    ),
],
    unroll=True,
    x=opps,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become a X person == How can I become a person who is not antonym(X)' 
desc = ''
test = MFT(**t, labels=0, name=name, capability = 'Negation',
          description=desc)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

## Coref

In [ ]:
t = editor.template(
    [(
        'If {male} and {female} were alone, do you think he would reject her?',
        'If {male} and {female} were alone, do you think she would reject him?',
    ),
        (
        'If {female} and {male} were alone, do you think he would reject her?',
        'If {female} and {male} were alone, do you think she would reject him?',
    )
    ],
    remove_duplicates=True, 
    nsamples=1000,
    unroll=True)
name = 'Simple coref: he and she'
desc = '' 
test = MFT(**t, labels=0, name=name, description=desc, capability='Coref')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
t = editor.template(
    [(
        'If {male} and {female} were married, would his family be happy?',
        'If {male} and {female} were married, would {female}\'s family be happy?',
    ),(
        'If {male} and {female} were married, would her family be happy?',
        'If {male} and {female} were married, would {male}\'s family be happy?',
    ),
    ]
        ,
    unroll=True,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Simple coref: his and her'
desc = '' 
test = MFT(**t, labels=0, name=name, description=desc, capability='Coref')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'Simple coref: his and her', 'Coref', 'TODO_DESCRIPTION')

## SRL

In [ ]:
print(', '.join(editor.suggest('Who is the best {mask} in the world?')))

In [ ]:
thing = ['chef', 'boxer', 'player', 'footballer', 'athlete', 'rapper', 'actor', 'singer', 'cook', 'magician', 'coach', 'cyclist', 'wrestler', 'drummer', 'musician', 'quarterback', 'hacker', 'baker', 'fighter', 'journalist', 'teacher', 'doctor', 'gamer', 'husband', 'DJ', 'person', 'man', 'woman', 'surgeon', 'comedian', 'trainer', 'programmer', 'guitarist', 'goalkeeper']

In [ ]:
print(', '.join(editor.suggest('Who do {mask} think is the the best {thing} in the world?', thing=thing)))

In [ ]:
subjects = ['you', 'people', 'readers', 'guys', 'fans', 'experts', 'scientists', 'Americans', 'students', 'men', 'voters', 'authors', 'conservatives', 'women', 'Canadians', 'analysts', 'critics', 'judges', 'artists', 'researchers', 'liberals', 'historians', 'Australians', 'journalists', 'Republicans', 'coaches', 'parents', 'kids', 'economists', 'reporters', 'consumers', 'veterans', 'doctors']

In [ ]:
print(', '.join(editor.suggest('Who do {subjects} think is the the {mask} {thing} in the world?', thing=thing, subjects=subjects)[:50]))

In [ ]:
best = ['best', 'greatest', 'worst', 'top', 'smartest', 'strongest', 'finests', 'happiest', 'coolest', 'richest', 'leading', 'brightest', 'premier', 'ultimate', 'dominant']

In [ ]:
t = editor.template((
    'Who do {subjects} think is the {best} {thing} in the world?',
    'Who is the {best} {thing} in the world according to {subjects}?'
),
    subjects=subjects,
    best=best,
    thing=thing,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Who do X think - Who is the ... according to X'
desc = '' 
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
print(', '.join([str(x) for x in editor.suggest('Are {mask} smaller than {a}?', a=['bananas', 'dogs', 'cars', 'cats', 'elephants'])][:100]))

In [ ]:
things = editor.suggest('Are {mask} smaller than {a}?',a=['bananas', 'dogs', 'cars', 'cats', 'elephants'] )[:100]

In [ ]:
print(', '.join([str(x) for x in editor.suggest('Are {a} {mask} than {a2}?', a=things)][:100]))

In [ ]:
comp = ['better', 'worse', 'cheaper', 'bigger', 'louder', 'longer', 'larger', 'smaller', 'warmer', 'colder', 'thicker', 'lighter', 'heavier']

Order doesn't matter for comparison

In [ ]:
t = editor.template([
    (
    'Are {t1} {comp} than {t2}?',
    'What is {comp}, {t2} or {t1}?'
    ),
    (
    'Are {t1} {comp} than {t2}?',
    'Are {t2} {comp} than {t1}?',
    ),
    (
    'Are {t1} {comp} than {t2}?',
    'What is {comp}, {t1} or {t2}?',
    )
]
    ,
    t = things,
    comp = comp,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'Order does not matter for comparison'
desc = '' 
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
# editor.template('Is {first_name1} {mask} {first_name2}?', remove_duplicates=True)[:100]

In [ ]:
print(', '.join(editor.suggest('Is {first_name1} {mask} to {first_name2}?', remove_duplicates=True)[:100]))
print()
print(', '.join(editor.suggest('Is {first_name1} {mask} {first_name2}?', remove_duplicates=True)[:100]))

In [ ]:
symmetric = ['dating', 'married to', 'close to', 'engaged to', 'connected to', 'married to', 'friends with', 'related to', 'an acquaintance of']

In [ ]:
t = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = symmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'Order does not matter for symmetric relations'
desc = 'e.g. dating, married to, close to, engaged to, etc' 
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Order matters for asymetric relations

In [ ]:
asymmetric = ['hurting', 'lying to', 'loyal to', 'faithful to', 'proposing to', 'indebted to', 'abusive to', 'using', 'expecting', 'beating', 'punching', 'raising', 'poisoning', 'protecting', 'kidnapping']

In [ ]:
t = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = asymmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'Order does matter for asymmetric relations'
desc = 'e.g. hurting lying to, faithful to, etc'
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

More traditional SRL

In [ ]:
print(', '.join(editor.suggest('Did John buy the {mask}?', remove_duplicates=True)[:100]))

In [ ]:
obj = ['farm', 'house', 'property', 'company', 'land', 'ticket', 'newspaper', 'book', 'island', 'estate', 'ranch', 'boat', 'horse', 'paper', 'business', 'gun', 'game', 'factory', 'castle', 'painting', 'rifle', 'car', 'school', 'building']

In [ ]:
print(', '.join(editor.suggest('Did John {mask} the {obj}?', obj=obj, remove_duplicates=True)[:100]))

In [ ]:
import pattern
import pattern.en
verbs = ['buy', 'purchase', 'sell', 'leave', 'own', 'take', 'keep', 'want', 'lose', 'destroy', 'inherit', 'find', 'use', 'need', 'receive', 'return', 'like', 'enjoy', 'abandon', 'manage', 'remember', 'miss', 'move', 'seize', 'steal']
a = pattern.en.tenses('stolen')[0]
verbs = [(v, pattern.en.conjugate(v, *a)) for v in verbs]
verbs[3] = ('leave', 'left')
verbs

In [ ]:
t = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was the {obj} {verb[1]} by {first_name}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'traditional SRL: active / passive swap'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
t = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was {first_name} {verb[1]} by the {obj}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'traditional SRL: wrong active / passive swap'
desc = ''
test = MFT(**t, labels=0, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

With people

In [ ]:
print(', '.join(editor.suggest('Does {first_name} {mask} {first_name2}?', remove_duplicates=True)[:100]))

In [ ]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
t = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name2} {verb[1]} by {first_name}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'traditional SRL: active / passive swap with people'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
t = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name} {verb[1]} by {first_name2}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'traditional SRL: wrong active / passive swap with people'
desc = ''
test = MFT(**t, labels=0, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

## Logic

In [ ]:
t = editor.template((
    'Is {first_name} {last_name} {a:p1} ?',
    'Is {first_name} {last_name} {a:p3}?',
),
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0)
# test.run(new_pp)
# test.summary(n=3)


In [ ]:
t = editor.template((
    'Is {first_name} {last_name} {a:p1} or {a:p2}?',
    'Is {first_name} {last_name} simultaneously {a:p3} and {a:p4}?',
),
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'A or B is not the same as C and D'
desc = ''
test = MFT(**t, labels=0, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)


In [ ]:
t = editor.template((
    'Is {first_name} {last_name} {a:p1} or {a:p2}?',
    'Is {first_name} {last_name} simultaneously {a:p1} and {a:p2}?',
),
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
name = 'A or B is not the same as A and B'
desc = ''
test = MFT(**t, labels=0, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:

t = editor.template((
    'Is {first_name} {last_name} {a:p1} {andor} {a:p2}?',
    'Is {first_name} {last_name} {a:p2} {andor} {a:p1}?',
),
    andor=['and', 'or'],
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'A and / or B is the same as B and / or A'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:

t = editor.template((
    'Is {first_name} {last_name} {a:nat} {p1}?',
    'Is {first_name} {last_name} {a:p1} and {nat}?',
),
    nat=editor.lexicons['nationality'][:20],
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'a {nationality} {profession} = a {profession} and {nationality}'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Reflexivity

In [ ]:
t = Perturb.perturb(list(all_questions), lambda x:(x, x), nsamples=1000, keep_original=False)
name = 'Reflexivity: (q, q) should be duplicate'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)
# test.summary(n=3)

Symmetry

In [ ]:
t = Perturb.perturb(qs, lambda x:(x[1], x[0]), nsamples=500, keep_original=True)
name = 'Symmetry: f(a, b) = f(b, a)'
desc = ''
test = INV(t.data, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [ ]:
# t.data[23]

In [ ]:
# labels

In [ ]:
import collections
def extract_unknown_implications(pairs, labels):
    graph = collections.defaultdict(lambda: set())
    ls = {}
    for x, y in zip(pairs, labels):
        graph[x[0]].add(x[1])
        graph[x[1]].add(x[0])
        t = tuple(sorted(x))
        ls[t] = y

    d = []
    l = []
    for x in graph:
        if len(graph[x]) == 1:
            continue
        for y in graph[x]:
            t = tuple(sorted((x, y)))
    #         print(t, ls[t])
        new = list(set([tuple(sorted(a)) for a in itertools.product(list(graph[x]), list(graph[x])) if a[0] != a[1]]))
        new = [a for a in new if a not in ls]
        for b, c in new:
            t1 = tuple(sorted((x, b)))
            t2 = tuple(sorted((x, c)))
            l1 = ls[t1]
            l2 = ls[t2]
            if l1 + l2 == 2:
                l3 = 1
            elif l1 + l2 == 1:
                l3 = 0
            else:
                continue
            new_x = [(x, b), (x, c), (b, c)]
            new_l = np.array([l1, l2, l3])
            d.append(new_x)
            l.append(new_l)
    return d, l

In [ ]:
data, ls = extract_unknown_implications(qs, labels)

In [ ]:
def expect_triplet(xs, preds, confs, labels, meta=None):
    if (preds[0] + preds[1]) == 2:
        if preds[2] != 1:
            return np.array([-3, -2, -1])
        else:
            return np.array([True, True, True])
    if (preds[0] + preds[1] == 1) and preds[1] != 0:
        if preds[1] != 0:
            return np.array([-3, -2, -1])
        else:
            return np.array([True, True, True])
    return None
#     if preds[0] != labels[0] or preds[1] != labels[1]:
#         return None
#     if preds[2] == labels[2]:
#         return np.array([True, True, True])
#     else:
#         return np.array([-3, -2, -1])
expect = Expect.testcase(expect_triplet)

In [ ]:
name = 'Testing implications'
desc = 'f(x, a) = 1 and f(x, b) = 1 => f(a, b) = 1\nf(x, a) = 1 and f(x, b) = 0 => f(a, b) = 0\n Only used (x, a, b) such that (x, a) and (x, b) in val dataset and (a, b) is not.\n Expectation function filters out examples where f(x, a) or f(x, b) are incorrect'
test = DIR(data, expect, labels=ls, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [127]:
suite.save('/home/marcotcr/tmp/qqp_suite.pkl')

## Bert

In [135]:
suite.run(new_pp, n=500, overwrite=True)

Running Modifier: adj
Predicting 500 examples
Running different adjectives
Predicting 500 examples
Running same adjectives, different people
Predicting 500 examples
Running same adjectives, different people v2
Predicting 500 examples
Running same adjectives, different people v3
Predicting 500 examples
Running Different animals
Predicting 500 examples
Running Irrelevant modifiers - animals
Predicting 500 examples
Running Irrelevant modifiers - people
Predicting 500 examples
Running Irrelevant preamble with different examples.
Predicting 500 examples
Running Preamble is relevant (different injuries)
Predicting 500 examples
Running How can I become more {synonym}?
Predicting 500 examples
Running add one typo
Predicting 1500 examples
Running contrations
Predicting 1407 examples
Running (q, paraphrase(q))
Predicting 18692 examples
Running Product of paraphrases(q1) * paraphrases(q2)
Predicting 9724 examples
Running Change same name in both questions
Predicting 5369 examples
Running Change s

In [136]:
suite.save('/tmp/bert')

In [137]:
suite.summary(n=3)

Vocabulary

Modifier: adj
Test cases:      999
Test cases run:  500
Fails (rate):    393 (78.6%)

Example fails:
0.9 ('Is Natalie Price a photographer?', 'Is Natalie Price an outstanding photographer?')
----
0.8 ('Is Victoria Ross an engineer?', 'Is Victoria Ross a honest engineer?')
----
1.0 ('Is Brittany White an investor?', 'Is Brittany White an exceptional investor?')
----


different adjectives
Test cases:      954
Test cases run:  500
Fails (rate):    5 (1.0%)

Example fails:
0.9 ('Is Zachary Phillips Jewish?', 'Is Zachary Phillips an immigrant?')
----
0.9 ('Is Ashley Ramirez American?', 'Is Ashley Ramirez Indian?')
----
0.6 ('Is Steven Hall American?', 'Is Steven Hall Jewish?')
----


same adjectives, different people
Test cases:      974
Test cases run:  500
Fails (rate):    31 (6.2%)

Example fails:
1.0 ('Is Daniel Cooper mad?', 'Is Samuel Johnson mad?')
----
1.0 ('Is Christina Gray white?', 'Is Isabella Johnson white?')
----
0.8 ('Is Lisa Williams racist?', 'Is Alyssa Miller 

## Roberta

In [132]:
suite.run(new_pp, n=500, seed=1, overwrite=True)

Running Modifier: adj
Predicting 500 examples
Running different adjectives
Predicting 500 examples
Running same adjectives, different people
Predicting 500 examples
Running same adjectives, different people v2
Predicting 500 examples
Running same adjectives, different people v3
Predicting 500 examples
Running Different animals
Predicting 500 examples
Running Irrelevant modifiers - animals
Predicting 500 examples
Running Irrelevant modifiers - people
Predicting 500 examples
Running Irrelevant preamble with different examples.
Predicting 500 examples
Running Preamble is relevant (different injuries)
Predicting 500 examples
Running How can I become more {synonym}?
Predicting 500 examples
Running add one typo
Predicting 1500 examples
Running contrations
Predicting 1434 examples
Running (q, paraphrase(q))
Predicting 18824 examples
Running Product of paraphrases(q1) * paraphrases(q2)
Predicting 9796 examples
Running Change same name in both questions
Predicting 5424 examples
Running Change s

In [133]:
suite.save('/tmp/roberta')

In [134]:
suite.summary(n=3)

Vocabulary

Modifier: adj
Test cases:      999
Test cases run:  500
Fails (rate):    381 (76.2%)

Example fails:
0.9 ('Is Jeffrey Young an editor?', 'Is Jeffrey Young an excellent editor?')
----
0.9 ('Is Nicole Campbell a player?', 'Is Nicole Campbell a fake player?')
----
1.0 ('Is Sophia Baker an actor?', 'Is Sophia Baker an active actor?')
----


different adjectives
Test cases:      954
Test cases run:  500
Fails (rate):    3 (0.6%)

Example fails:
0.9 ('Is Ethan Russell American?', 'Is Ethan Russell an immigrant?')
----
0.8 ('Is Emma Ortiz Indian?', 'Is Emma Ortiz an immigrant?')
----
0.9 ('Is Ethan Russell American?', 'Is Ethan Russell an immigrant?')
----


same adjectives, different people
Test cases:      974
Test cases run:  500
Fails (rate):    0 (0.0%)


same adjectives, different people v2
Test cases:      989
Test cases run:  500
Fails (rate):    3 (0.6%)

Example fails:
0.9 ('Is Alexis Diaz an astronaut?', 'Is Alyssa Diaz an astronaut?')
----
0.9 ('Is Alexis Diaz an astro